## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용
- 무선청소기 검색에 사용할 URL
    https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

    한페이지가 로드될때까지 5초정도 필요

In [28]:
## 필수 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib import parse
from tqdm import tqdm

In [4]:
## 웹드라이버로 크롬오픈
import time


driver = webdriver.Chrome()
url = r'https://search.danawa.com/dsearch.php?query=무선청소기&page=2&limit=120&sort=saveDESC&list=list'
driver.get(url)
time.sleep(5.0)

In [5]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [6]:
soup

<html class="js no-touch" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>무선청소기 : 다나와 통합검색</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="무선청소기 통합검색 : 다나와 통합검색" name="title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." name="description"/>
<meta content="무선청소기, 다나와, 최저가, 가격비교, 검색, 통합검색, 차이슨 무선청소기, 디베아 무선청소기, 삼성 무선청소기, 일렉트로룩스 무선청소기, 샤크 무선청소기, 다이슨 무선청소기, 테팔무선청소기, 샤오미 무선청소기, lg 무선청소기, 유선청소기" name="keywords"/>
<meta content="무선청소기 통합검색 : 다나와 통합검색" property="og:title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." property="og:description"/>
<meta content="https://img.danawa.com/new/tour/img/logo/sns_danawa.jpg" property="og:image"/>
<meta content="width=1280" name="viewport"/>
<link href="//img.danawa.com/new/danawa_main/v1/img/danawa_favicon.ico" rel="shortcut icon"/>
<link href="//static.danawa.com/css/common.css?v=2402280851" rel="stylesheet" type="text/css"/>
<link hre

In [7]:
prodItems = soup.select('ul.product_list > li.prod_item')


In [8]:
# id="productItem15155642" 식으로 전개되므로 productItem이 아니면 패스
# 제품명 가져오기, 만약 애드스마트 영역이면 예외발생
prodItems[0].select('p.prod_name > a')[0].text.strip()

'에어메이드 스마트포스 AMC-3500'

In [9]:
# 제품스펙 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t', '')

'핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 45분(최대) / 충전시간: 5시간 / 분리형(1개) / 리튬이온 / 2500mAh / [청소] LED라이트 / UV살균 / [부가] 디스플레이표시 / 헤파필터 / 5단계여과 / 무게: 2.5kg / 크기(가로x세로x깊이): 255x1230x215mm'

In [10]:
# 제품가격 가져오기
# 책에서는 1위 데이터를 가져오도록 했음
# 우리는 input 중 두번째 값이 최소값을 담고 있는 것을 확인
prodItems[0].select('input')[1].get('value')

'197000'

In [11]:
# 제품 인트로
prodItems[2].select('p.intro_text')[0].text

'배터리 2개+펫브러쉬 제외 액세서리 6종 구성 (청소기+청정스테이션 합포장)'

##### 1번 페이지에서 상품정보 추출

In [12]:
prodData = []

for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip() # 상품명 가져오기
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 정보 가져오기
        price = prodItem.select('input')[1].get('value') # 최저가 가져오기
        prodData.append([prodName, specList, price])
        
    except:
        pass

In [13]:
# 결과 확인
prodData

[['에어메이드 스마트포스 AMC-3500',
  '핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 370W / 2022년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 45분(최대) / 충전시간: 5시간 / 분리형(1개) / 리튬이온 / 2500mAh / [청소] LED라이트 / UV살균 / [부가] 디스플레이표시 / 헤파필터 / 5단계여과 / 무게: 2.5kg / 크기(가로x세로x깊이): 255x1230x215mm',
  '197000'],
 ['일렉트로룩스 Well Q8 WQ81-3OMBF',
  '핸디스틱청소기 / 무선 / 흡입형 / 2021년형 / [구성] 충전 / 스탠드거치 / 브러쉬: 바닥 / 솔형 / 틈새 / [배터리] 사용시간: 53분(최대) / 충전시간: 4시간30분 / 내장형 / [청소] 싸이클론흡입 / LED라이트 / 먼지통: 0.3L / [부가] 셀프스탠딩 / 헤파필터 / 5단계여과 / 색상: 마호가니브론즈 / 무게: 2.8kg / 크기(가로x세로x깊이): 258ㅌ1105ㅌ140mm',
  '329000'],
 ['삼성전자 비스포크 제트 VS20A957E3',
  '핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 210W / 2021년형 / [구성] 거치대: 청정스테이션 / 먼지비움 / 충전 / 스탠드거치 / 먼지비움시간: 20초 / 브러쉬: 바닥 / 물걸레: 일반 / 침구 / 솔형 / 틈새 / 연장관 / 먼지봉투: 1.2L / [배터리] 사용시간: 1시간(최대) / 충전시간: 3시간30분 / 분리형(2개) / 리튬이온 / 2200+2200mAh / 25.2V / [청소] 싸이클론흡입 / 자동물공급 / 물분사 / 디지털인버터모터 / 먼지통: 0.5L / [부가] 대화형알림창 / 디스플레이표시 / 배터리잔량표시 / 워셔블헤파필터 / 5단계여과 / 색상: 미스티화이트 / 무게: 2.5kg / 크기(가로x세로x깊이): 250x930x202mm',
  '592730'],
 ['쿠

##### 여러 페이지 크롤링 하는 방법

- URL
    https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

In [26]:
## 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
            f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
            'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
            'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [38]:
## 상품정보 추출하는 함수
def getProdItems(prodItem):
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip() # 상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '') # 상품 정보 가져오기
            price = prodItem.select('input')[1].get('value') # 최저가 가져오기
            prodData.append([prodName, specList, price])
            
        except:
            pass
    return prodData

In [27]:
# 검색어에 ' '가 있으면 URL에서 링크 클릭이 안되는 문제 발생 --> urlEncode : parse.quote()
getSearchPageUrl('닌텐도 스위치', 3)

'https://search.danawa.com/dsearch.php?query=%EB%8B%8C%ED%85%90%EB%8F%84%20%EC%8A%A4%EC%9C%84%EC%B9%98&originalQuery=%EB%8B%8C%ED%85%90%EB%8F%84%20%EC%8A%A4%EC%9C%84%EC%B9%98&previousKeyword=%EB%8B%8C%ED%85%90%EB%8F%84%20%EC%8A%A4%EC%9C%84%EC%B9%98&checkedInfo=N&volumeType=allvs&page=3&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'

In [34]:
total_page = 10

for page in tqdm(range(1, total_page+1)):
    # print('페이지 처리') # tqdm 진행바 여러번 출력
    time.sleep(1)

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [47]:
## 여러 페이지 검색 후 크롤링 하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초 정도 대기
driver.implicitly_wait(3.0)

keyword = '무선청소기'
totalPage = 20
prodDataTotal = [] # 최종적으로 저장할 리스트

for page in tqdm(range(11, totalPage+1)):
    # 검색 페이지 이동
    url = getSearchPageUrl(keyword, page)
    driver.get(url)
    # 페이지 로딩이 완료될 때까지 5초간 대기
    time.sleep(5.0)

    # 현재 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList =getProdItems(prodItems) # 리스트로 추출하는 함수

    # 추출된 정보를 prodDataTotal 추가
    prodDataTotal += prodItemList


100%|██████████| 10/10 [01:59<00:00, 11.90s/it]


In [48]:
dfProdData = pd.DataFrame(prodDataTotal)

In [49]:
dfProdData.columns = ['제품명', '제품스펙', '최저가']

In [52]:
dfProdData.to_excel('./data/다나와_무선청소기_결과2.xlsx', index = False)